# dice (Dice)

In [ ]:
//// test

open testing

## sixth_power_sequence

In [ ]:
inl sixth_power_sequence () =
    stream.iterate_map ((*) 6) (fun x => if x <= 0 then None else Some x) 1

In [ ]:
//// test

sixth_power_sequence ()
|> stream.take_while (fun _ i => i <= 7i32)
|> stream.to_list
|> _assert_eq [ 1u64; 6; 36; 216; 1296; 7776; 46656; 279936 ]

{ name = __assert_eq; actual = UH0_1
  (1UL,
   UH0_1
     (6UL,
      UH0_1
        (36UL,
         UH0_1
           (216UL,
            UH0_1
              (1296UL, UH0_1 (7776UL, UH0_1 (46656UL, UH0_1 (279936UL, UH0_0)))))))); expected = UH0_1
  (1UL,
   UH0_1
     (6UL,
      UH0_1
        (36UL,
         UH0_1
           (216UL,
            UH0_1
              (1296UL, UH0_1 (7776UL, UH0_1 (46656UL, UH0_1 (279936UL, UH0_0)))))))) }


## accumulate_dice_rolls

In [ ]:
inl accumulate_dice_rolls
    (rolls : list u8)
    (power : i8)
    acc
    =
    inl rec body acc power rolls =
        match rolls with
        | _ when power < 0 =>
            inl result = acc + 1
            trace Debug
                fun () => "dice.accumulate_dice_rolls"
                fun () => { power acc result }
            Some (result, rolls)
        | [] => None
        | roll :: rest when roll > 1 =>
            inl coeff = sixth_power_sequence () |> stream.item power
            inl value = conv (roll - 1) * coeff
            trace Debug
                fun () => "dice.accumulate_dice_rolls"
                fun () => { power acc roll value }
            loop (acc + value) (power - 1) rest
        | roll :: rest =>
            trace Debug
                fun () => "dice.accumulate_dice_rolls"
                fun () => { power acc roll }
            loop acc (power - 1) rest
    and inl loop acc power rolls =
        if var_is rolls |> not
        then body acc power rolls
        else
            inl acc = dyn acc
            join body acc power rolls
    loop acc power rolls

In [ ]:
//// test

accumulate_dice_rolls [ 6; 5; 4; 3; 2 ] 0 1000i32
|> _assert_eq (Some (1006, [ 5; 4; 3; 2 ]))

00:00:00 d #1 dice.accumulate_dice_rolls / { power = 0; acc = 1000; roll = 6; value = 5 }
00:00:00 d #2 dice.accumulate_dice_rolls / { power = -1; acc = 1005; result = 1006 }
{ name = __assert_eq; actual = US6_0 (1006, UH0_1 (5uy, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0))))); expected = US6_0 (1006, UH0_1 (5uy, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0))))) }


In [ ]:
//// test

accumulate_dice_rolls [ 6; 5; 4; 3; 2 ] 1 1000i32
|> _assert_eq (Some (1035, [ 4; 3; 2 ]))

00:00:00 d #1 dice.accumulate_dice_rolls / { power = 1; acc = 1000; roll = 6; value = 30 }
00:00:00 d #2 dice.accumulate_dice_rolls / { power = 0; acc = 1030; roll = 5; value = 4 }
00:00:00 d #3 dice.accumulate_dice_rolls / { power = -1; acc = 1034; result = 1035 }
{ name = __assert_eq; actual = US6_0 (1035, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0)))); expected = US6_0 (1035, UH0_1 (4uy, UH0_1 (3uy, UH0_1 (2uy, UH0_0)))) }


In [ ]:
//// test

accumulate_dice_rolls [ 6; 5; 4; 3; 2 ] 2 1000i32
|> _assert_eq (Some (1208, [ 3; 2 ]))

00:00:00 d #1 dice.accumulate_dice_rolls / { power = 2; acc = 1000; roll = 6; value = 180 }
00:00:00 d #2 dice.accumulate_dice_rolls / { power = 1; acc = 1180; roll = 5; value = 24 }
00:00:00 d #3 dice.accumulate_dice_rolls / { power = 0; acc = 1204; roll = 4; value = 3 }
00:00:00 d #4 dice.accumulate_dice_rolls / { power = -1; acc = 1207; result = 1208 }
{ name = __assert_eq; actual = US6_0 (1208, UH0_1 (3uy, UH0_1 (2uy, UH0_0))); expected = US6_0 (1208, UH0_1 (3uy, UH0_1 (2uy, UH0_0))) }


## roll_within_bounds

In [ ]:
inl roll_within_bounds max rolls =
    inl power = listm.length rolls - 1
    match accumulate_dice_rolls rolls power 0 with
    | Some (result, _) when result >= 1 && result <= max => Some result
    | _ => None

In [ ]:
//// test

roll_within_bounds 2000i32 [ 1; 5; 4; 4; 5 ]
|> _assert_eq (Some 995)

00:00:00 d #1 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 1 }
00:00:00 d #2 dice.accumulate_dice_rolls / { power = 3; acc = 0; roll = 5; value = 864 }
00:00:00 d #3 dice.accumulate_dice_rolls / { power = 2; acc = 864; roll = 4; value = 108 }
00:00:00 d #4 dice.accumulate_dice_rolls / { power = 1; acc = 972; roll = 4; value = 18 }
00:00:00 d #5 dice.accumulate_dice_rolls / { power = 0; acc = 990; roll = 5; value = 4 }
00:00:00 d #6 dice.accumulate_dice_rolls / { power = -1; acc = 994; result = 995 }
{ name = __assert_eq; actual = US6_0 995; expected = US6_0 995 }


In [ ]:
//// test

roll_within_bounds 2000i32 [ 2; 2; 6; 4; 5 ]
|> _assert_eq (Some 1715)

00:00:00 d #1 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 2; value = 1296 }
00:00:00 d #2 dice.accumulate_dice_rolls / { power = 3; acc = 1296; roll = 2; value = 216 }
00:00:00 d #3 dice.accumulate_dice_rolls / { power = 2; acc = 1512; roll = 6; value = 180 }
00:00:00 d #4 dice.accumulate_dice_rolls / { power = 1; acc = 1692; roll = 4; value = 18 }
00:00:00 d #5 dice.accumulate_dice_rolls / { power = 0; acc = 1710; roll = 5; value = 4 }
00:00:00 d #6 dice.accumulate_dice_rolls / { power = -1; acc = 1714; result = 1715 }
{ name = __assert_eq; actual = US6_0 1715; expected = US6_0 1715 }


In [ ]:
//// test

roll_within_bounds 2000i32 [ 4; 1; 1; 2; 3 ]
|> _assert_eq None

00:00:00 d #1 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 4; value = 3888 }
00:00:00 d #2 dice.accumulate_dice_rolls / { power = 3; acc = 3888; roll = 1 }
00:00:00 d #3 dice.accumulate_dice_rolls / { power = 2; acc = 3888; roll = 1 }
00:00:00 d #4 dice.accumulate_dice_rolls / { power = 1; acc = 3888; roll = 2; value = 6 }
00:00:00 d #5 dice.accumulate_dice_rolls / { power = 0; acc = 3894; roll = 3; value = 2 }
00:00:00 d #6 dice.accumulate_dice_rolls / { power = -1; acc = 3896; result = 3897 }
{ name = __assert_eq; actual = US6_1; expected = US6_1 }


## calculate_dice_count

In [ ]:
inl calculate_dice_count max =
    inl rec body n p =
        inl return () =
            trace Debug
                fun () => "dice.calculate_dice_count"
                fun () => { max p n }
            n
        if p < max then
            inl p' = p * 6
            if p' > p
            then loop (n + 1) p'
            else return ()
        else return ()
    and inl loop n p =
        if var_is max |> not
        then body n p
        else
            inl n = dyn n
            inl p = dyn p
            join body n p
    if max = 1
    then 1
    else loop 0 1

In [ ]:
//// test

calculate_dice_count 36i32
|> _assert_eq 2i32

00:00:00 d #1 dice.calculate_dice_count / { max = 36; p = 36; n = 2 }
{ name = __assert_eq; actual = 2; expected = 2 }


In [ ]:
//// test

calculate_dice_count 7777i32
|> _assert_eq 6i32

00:00:00 d #1 dice.calculate_dice_count / { max = 7777; p = 46656; n = 6 }
{ name = __assert_eq; actual = 6; expected = 6 }


## roll_dice

In [ ]:
let roll_dice () : u8 =
    random' 1 7

## rotate_number

In [ ]:
inl rotate_number forall t {number}. (max : i64) (n : t) : t =
    (conv n - 1 + max) % max + 1 |> conv

## rotate_numbers

In [ ]:
inl rotate_numbers max items =
    items |> stream.map (rotate_number max)

In [ ]:
//// test

listm'.init_series -1 14 1i32
|> stream.from_list
|> rotate_numbers 6
|> stream.to_list
|> _assert_eq [ 5; 6; 1; 2; 3; 4; 5; 6; 1; 2; 3; 4; 5; 6; 1; 2 ]

{ name = __assert_eq; actual = UH0_1
  (5,
   UH0_1
     (6,
      UH0_1
        (1,
         UH0_1
           (2,
            UH0_1
              (3,
               UH0_1
                 (4,
                  UH0_1
                    (5,
                     UH0_1
                       (6,
                        UH0_1
                          (1,
                           UH0_1
                             (2,
                              UH0_1
                                (3,
                                 UH0_1
                                   (4,
                                    UH0_1
                                      (5, UH0_1 (6, UH0_1 (1, UH0_1 (2, UH0_0)))))))))))))))); expected = UH0_1
  (5,
   UH0_1
     (6,
      UH0_1
        (1,
         UH0_1
           (2,
            UH0_1
              (3,
               UH0_1
                 (4,
                  UH0_1
                    (5,
                     UH0_1
                       (6,
                 

## create_sequential_roller

In [ ]:
inl create_sequential_roller list =
    inl s = list |> listm.rev |> listm.append list |> stream.from_list |> stream.memoize
    inl current_index = mut 0i64
    inl acc = mut 1
    inl len = mut -1
    inl last_item = mut None
    let rec loop () =
        fun () =>
            inl current_index = *current_index
            inl acc = *acc
            inl len = *len
            inl last_item = *last_item |> optionm'.box
            trace Debug
                fun () => "dice.create_sequential_roller / roll"
                fun () => { current_index acc len last_item = last_item |> sm'.format_debug }
        |> fun x => x ()
        match s () |> stream.try_item *current_index with
        | Some item =>
            current_index <- *current_index + 1
            last_item <- Some item
            item
        | None =>
            trace Debug (fun () => "dice.create_sequential_roller / roll / None") id
            if *len = -1
            then len <- *current_index
            acc <-
                if *acc >= *len
                then 1
                else *acc + 1
            current_index <- *acc - 1
            last_item <- None
            loop ()
    loop

In [ ]:
//// test

inl sequential_roll = create_sequential_roller [ 1i32; 2; 3; 4 ]

am.init 50i32 (ignore >> sequential_roll)
|> _assert_eq (a ;[ 1; 2; 3; 4; 4; 3; 2; 1; 2; 3; 4; 4; 3; 2; 1; 3; 4; 4; 3; 2; 1; 4; 4; 3; 2; 1;
4; 3; 2; 1; 3; 2; 1; 2; 1; 1; 1; 2; 3; 4; 4; 3; 2; 1; 2; 3; 4; 4; 3; 2 ] : a i32 i32)

00:00:00 d #1 dice.create_sequential_roller / roll / { current_index = 0; acc = 1; len = -1; last_item = None }
00:00:00 d #2 dice.create_sequential_roller / roll / { current_index = 1; acc = 1; len = -1; last_item = Some 1 }
00:00:00 d #3 dice.create_sequential_roller / roll / { current_index = 2; acc = 1; len = -1; last_item = Some 2 }
00:00:00 d #4 dice.create_sequential_roller / roll / { current_index = 3; acc = 1; len = -1; last_item = Some 3 }
00:00:00 d #5 dice.create_sequential_roller / roll / { current_index = 4; acc = 1; len = -1; last_item = Some 4 }
00:00:00 d #6 dice.create_sequential_roller / roll / { current_index = 5; acc = 1; len = -1; last_item = Some 4 }
00:00:00 d #7 dice.create_sequential_roller / roll / { current_index = 6; acc = 1; len = -1; last_item = Some 3 }
00:00:00 d #8 dice.create_sequential_roller / roll / { current_index = 7; acc = 1; len = -1; last_item = Some 2 }
00:00:00 d #9 dice.create_sequential_roller / roll / { current_index = 8; acc = 1; len = -

## roll_progressively

In [ ]:
inl roll_progressively roll reroll max =
    inl power = (calculate_dice_count max) - 1
    let rec loop rolls size =
        if size < power + 1
        then loop (roll () :: rolls) (size + 1)
        else accumulate_dice_rolls rolls power 0 |> function
            | Some (result, _) when result <= max => result
            | _ when reroll => loop (listm.init power (fun _ => roll ())) power
            | _ => loop (roll () :: rolls) (size + 1)
    loop [] 0

In [ ]:
//// test

roll_progressively roll_dice false 1
|> _assert_eq 1i32

00:00:00 d #1 dice.accumulate_dice_rolls / { power = 0; acc = 0; roll = 6; value = 5 }
00:00:00 d #2 dice.accumulate_dice_rolls / { power = -1; acc = 5; result = 6 }
00:00:00 d #3 dice.accumulate_dice_rolls / { power = 0; acc = 0; roll = 6; value = 5 }
00:00:00 d #4 dice.accumulate_dice_rolls / { power = -1; acc = 5; result = 6 }
00:00:00 d #5 dice.accumulate_dice_rolls / { power = 0; acc = 0; roll = 1 }
00:00:00 d #6 dice.accumulate_dice_rolls / { power = -1; acc = 0; result = 1 }
{ name = __assert_eq; actual = 1; expected = 1 }


In [ ]:
//// test

inl sequential_roll = create_sequential_roller [ 5; 4; 4; 5; 1 ]

roll_progressively sequential_roll false 2000i32
|> _assert_eq 995

00:00:00 d #1 dice.calculate_dice_count / { max = 2000; p = 7776; n = 5 }
00:00:00 d #2 dice.create_sequential_roller / roll / { current_index = 0; acc = 1; len = -1; last_item = None }
00:00:00 d #3 dice.create_sequential_roller / roll / { current_index = 1; acc = 1; len = -1; last_item = Some 5uy }
00:00:00 d #4 dice.create_sequential_roller / roll / { current_index = 2; acc = 1; len = -1; last_item = Some 4uy }
00:00:00 d #5 dice.create_sequential_roller / roll / { current_index = 3; acc = 1; len = -1; last_item = Some 4uy }
00:00:00 d #6 dice.create_sequential_roller / roll / { current_index = 4; acc = 1; len = -1; last_item = Some 5uy }
00:00:00 d #7 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 1 }
00:00:00 d #8 dice.accumulate_dice_rolls / { power = 3; acc = 0; roll = 5; value = 864 }
00:00:00 d #9 dice.accumulate_dice_rolls / { power = 2; acc = 864; roll = 4; value = 108 }
00:00:00 d #10 dice.accumulate_dice_rolls / { power = 1; acc = 972; roll = 4; value = 18 }
00:

In [ ]:
//// test

inl sequential_roll = create_sequential_roller [ 5; 4; 4; 5; 2 ]

roll_progressively sequential_roll false 2000i32
|> _assert_eq 1678

00:00:00 d #1 dice.calculate_dice_count / { max = 2000; p = 7776; n = 5 }
00:00:00 d #2 dice.create_sequential_roller / roll / { current_index = 0; acc = 1; len = -1; last_item = None }
00:00:00 d #3 dice.create_sequential_roller / roll / { current_index = 1; acc = 1; len = -1; last_item = Some 5uy }
00:00:00 d #4 dice.create_sequential_roller / roll / { current_index = 2; acc = 1; len = -1; last_item = Some 4uy }
00:00:00 d #5 dice.create_sequential_roller / roll / { current_index = 3; acc = 1; len = -1; last_item = Some 4uy }
00:00:00 d #6 dice.create_sequential_roller / roll / { current_index = 4; acc = 1; len = -1; last_item = Some 5uy }
00:00:00 d #7 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 2; value = 1296 }
00:00:00 d #8 dice.accumulate_dice_rolls / { power = 3; acc = 1296; roll = 5; value = 864 }
00:00:00 d #9 dice.accumulate_dice_rolls / { power = 2; acc = 2160; roll = 4; value = 108 }
00:00:00 d #10 dice.accumulate_dice_rolls / { power = 1; acc = 2268; roll = 

In [ ]:
//// test

inl { level } = get_trace_state_or_init None
level <- Info

join dyn listm'.init_series 1i32 10000 1
|> listm.map (fun _ => roll_progressively roll_dice false 10i32)
|> listm'.group_by id
|> listm.map (fun (k, v) => k, (listm.length v : i32))
|> listm'.box
|> listm'.to_array'
|> fun ar =>
    $'!ar |> Array.sortBy (fun (struct (a, b)) -> a)' : a i32 $'struct (int32 * int32)'

index value 0 (1, 263) Item1 1 Item2 263 1 (2, 712) Item1 2 Item2 712 2 (3, 1396) Item1 3 Item2 1396 3 (4, 1320) Item1 4 Item2 1320 4 (5, 1362) Item1 5 Item2 1362 5 (6, 1362) Item1 6 Item2 1362 6 (7, 256) Item1 7 Item2 256 7 (8, 718) Item1 8 Item2 718 8 (9, 1323) Item1 9 Item2 1323 9 (10, 1288) Item1 10 Item2 1288

In [ ]:
//// test

inl { level } = get_trace_state_or_init None
level <- Info

join dyn listm'.init_series 1i32 10000 1
|> listm.map (fun _ => roll_progressively roll_dice true 10i32)
|> listm'.group_by id
|> listm.map (fun (k, v) => k, (listm.length v : i32))
|> listm'.box
|> listm'.to_array'
|> fun ar =>
    $'!ar |> Array.sortBy (fun (struct (a, b)) -> a)' : a i32 $'struct (int32 * int32)'

index value 0 (1, 1008) Item1 1 Item2 1008 1 (2, 976) Item1 2 Item2 976 2 (3, 1014) Item1 3 Item2 1014 3 (4, 1026) Item1 4 Item2 1026 4 (5, 962) Item1 5 Item2 962 5 (6, 1023) Item1 6 Item2 1023 6 (7, 1015) Item1 7 Item2 1015 7 (8, 1023) Item1 8 Item2 1023 8 (9, 944) Item1 9 Item2 944 9 (10, 1009) Item1 10 Item2 1009

In [ ]:
//// test
//// timeout=180000

inl { level } = get_trace_state_or_init None
level <- Info

join dyn listm'.init_series 1i32 100 1
|> listm.iter fun n =>
    listm'.init_series 0i32 1 1
    |> listm.iter fun reroll =>
        join dyn listm'.init_series 1i32 3500 1
        |> listm.map fun _ => roll_progressively roll_dice (reroll = 1) n
        |> listm'.group_by id
        |> listm.length
        |> __assert_eq Silent n

## main

In [ ]:
inl main (_args : array_base string) =
    inl result = roll_progressively roll_dice true (limit.max : i64)
    trace Debug
        fun () => "dice.main"
        fun () => { result }
    0i32

inl main () =
    inl rotate_numbers' : i64 -> stream.stream u8 -> stream.stream u8 =
        rotate_numbers

    inl create_sequential_roller' :
        list u8 
        -> (() -> u8)
        =
        create_sequential_roller

    inl roll_progressively' :
        (() -> u8)
        -> bool
        -> u64
        -> u64
        =
        roll_progressively

    inl roll_within_bounds' :
        u64
        -> list u8
        -> optionm'.option' u64
        =
        fun a b =>
            roll_within_bounds a b
            |> optionm'.box

    $'let rotate_numbers x = !rotate_numbers' x' : ()
    $'let create_sequential_roller x = !create_sequential_roller' x' : ()
    $'let roll_progressively x = !roll_progressively' x' : ()
    $'let roll_within_bounds x = !roll_within_bounds' x' : ()
    $'let main args = !main args' : ()